In [1]:
import numpy as np
import random
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from pylab import rcParams
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'
rcParams['font.family'] = 'serif'
rc('text.latex',preamble=r'\usepackage{/Users/kevinlevy/Documents/codes/master_thesis/code/apjfonts}')
from tqdm import tqdm
import lensing_estimator
from cosmo import CosmoCalc
import lensing
import foregrounds as fg
import experiments as exp
import sims
import hlc
import stats
import tools

KeyboardInterrupt: 

In [ ]:
nber_maps = 1
mapparams = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
cmb_map = sims.cmb_mock_data(nber_maps, mapparams, l, cl)[0]

In [ ]:
_, extent = tools.make_grid(mapparams)
plt.figure(figsize = (6,6))
plt.imshow(cmb_map, extent = extent, cmap = cm.RdBu_r)
plt.colorbar(shrink = 0.825).set_label(label=r'$T \ [\mu$'+'K]', size=15)
plt.xlabel(r'$\theta_x\ $'+'[arcmin]', fontsize = '15')
plt.ylabel(r'$\theta_y\ $'+'[arcmin]', fontsize = '15')
plt.savefig('cmb_realization.eps', dpi = 200., bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
nber_clus = 3000
nber_rand = 30000
nber_cov = 200
map_params = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.4)
l, nl = exp.noise_power_spectrum(noiseval_white = 2.0)
l, nl_deconvolved =  exp.noise_power_spectrum(noiseval_white = 2.0, beam_fwhm = 1.4)
cl_noise =  nl_deconvolved         

In [ ]:
covariance_matrix2, correlation_matrix = lensing_estimator.covariance_matrix(nber_clus, nber_rand, nber_cov, map_params,  l, cl, bl = bl, nl = nl, cl_noise = cl_noise)




In [ ]:
with open('covariance_matrix.npy', 'rb') as file:
    covariance_matrix = np.load(file)   

In [ ]:
plt.imshow(covariance_matrix1)
plt.colorbar()
plt.subplots()
plt.imshow(covariance_matrix2)
plt.colorbar()
plt.subplots()
plt.imshow(covariance_matrix)
plt.colorbar()
plt.subplots()
plt.imshow(covariance_matrix2-covariance_matrix)
plt.colorbar()